In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.append('../../wildlife-tools')
sys.path.append('..')

import timm
import torch
import torchvision.transforms as T
import numpy as np
import pandas as pd

from wildlife_datasets import datasets, loader
from wildlife_tools.data import WildlifeDataset
from wildlife_tools.features import DeepFeatures
from PIL import Image
from urllib.request import urlopen

In [ ]:
model = timm.create_model("hf-hub:BVRA/MegaDescriptor-L-384", pretrained=True)
model = model.eval()
root_dataset = '../data/'
root_dataframe = '../data/_dataframes/'

In [ ]:
transform = T.Compose([T.Resize(size=(384, 384)),
                              T.ToTensor(), 
                              T.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])]) 

In [ ]:
for d_name in datasets.names_all[11:]:
    d = loader.load_dataset(d_name, root_dataset, root_dataframe)
    if len(d.df) > 10000:
        print(len(d.df), d)
        with torch.no_grad():
            dataset = WildlifeDataset(d.df, d.root, transform=transform)
            extractor = DeepFeatures(model, device='cuda', batch_size=32)
            output = extractor(dataset)
            np.save(f'../data/_features/features_{d_name.__name__}.npy', output)